In [ ]:
import os
import warnings


In [ ]:
# os.environ["PYTHONWARNINGS"]="ignore"
# os.environ["PYTORCH_ENABLE_MPS_FALLBACK"]="1"
warnings.filterwarnings("ignore")
# os.environ

In [ ]:

import glob

import numpy as np
# from ray.tune.search.hyperopt import HyperOptSearch

import pandas as pd

from neuralforecast.core import NeuralForecast

from statsforecast import StatsForecast

from utils.utils import build_market_image
from configs import futures_defaults as defaults
# optuna.logging.set_verbosity(optuna.logging.WARNING) # Use this to disable training prints from optuna

In [ ]:
COIN_PAIRS=defaults.COIN_PAIRS
target_pair=defaults.target_pair
time_frame=defaults.time_frame
DATA_DIR=defaults.DATA_DIR
model_path=defaults.forecasting_model_path

In [ ]:
data=build_market_image(target_pair=target_pair,time_frame='1h',axis=0,dir='data_forex')


In [ ]:

model=NeuralForecast.load(model_path)

In [ ]:
front=['y','ds','symbol']
data=data[front+[c for c in data.columns if c not in front]]
data=data[front]

In [ ]:
data.head(5)

In [ ]:
model.fit(data,id_col='symbol')

In [ ]:

model.save(model_path,overwrite=True,save_dataset=False)

In [ ]:

context_length=max([sub_model.hparams['input_size'] for sub_model in model.models])
context_length

In [ ]:
eth_data=data[data['symbol']==target_pair].copy()
# data=eth_data
split=context_length

test=data.groupby('symbol').tail(split)
test.groupby('symbol').apply(lambda x: len(x))



In [ ]:
preds=model.predict(test)

In [ ]:
StatsForecast.plot(test,preds,id_col='symbol', engine='matplotlib')
